In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
struct_dir = '/home/eva/Documents/designs/p-IL-dimer/simulation-input/'
pdb_file = struct_dir+'p-IL_fromtleap.pdb'
prmtop_file = struct_dir+'p-IL.prmtop'
inpcrd_file = struct_dir+'p-IL.inpcrd'

In [ ]:
prot = md.load_pdb(pdb_file)

chain1 = [atom.index for atom in prot.topology.chain(0).atoms] #chain1
chain2 = [atom.index for atom in prot.topology.chain(1).atoms] #chain2
assembly = [atom.index for atom in prot.topology.atoms] #full assembly

chain1_top = md.load_pdb(pdb_file, atom_indices=chain1)

chain2_top = md.load_pdb(pdb_file, atom_indices=chain2)

In [ ]:
coord = md.load(inpcrd_file, top = prmtop_file)

phi_array = md.compute_phi(coord)
psi_array = md.compute_psi(coord)

phi_atoms = phi_array[0].tolist()
psi_atoms = psi_array[0].tolist()

phi_atoms_res_5_25 = phi_atoms[4:25]+phi_atoms[34:55] # needs to be generalised depending on length of monomer helix
psi_atoms_res_5_25 = psi_atoms[4:25]+psi_atoms[34:55]

In [ ]:
# calculating end-to-end distance of second monomer to estimate size of the funnel

chain2_res = [res.name for res in chain2_top.topology.residues]

if 'ACE' in chain2_res: 
    chain2_length = len(chain2_res-2) # exluding Ac and NH2 caps (they don't have CA atoms) from the end-to-end calculation
    first_res_idx = 1
else:
    chain2_length = len(chain2_res)
    first_res_idx = 0

first_ca = [atom.index for atom in chain2_top.topology.atoms if ((atom.residue.index == first_res_idx) and (atom.name == 'CA'))]
last_ca = [atom.index for atom in chain2_top.topology.atoms if ((atom.residue.index == chain2_length-1) and (atom.name == 'CA'))]

ca_array = np.array([[first_ca[0], last_ca[0]], [0,0]])
end_to_end = md.compute_distances(chain2_top, ca_array)
end_to_end_chain2 = float(end_to_end[0][0])

funnel_wall = end_to_end_chain2/2.0 + 1 # funnel wall half the length of the monomer + add 10 Angstrom padding
# the funnel wall will need to be split in h+f (wall width + wall buffer)

In [ ]:
#average alpha helix phi and psi dihedrals
phi_avg = (-57.0*np.pi)/180.0
psi_avg = (-47.0*np.pi)/180.0

#Ramachandran plot region for right-handed alpha-helix (in deg)
phi_min = -130.0
phi_max = -30.0
psi_min = -68.0
psi_max = 30.0

phi0 = (phi_min +phi_max)/2.0
psi0 = (psi_min+psi_max)/2.0
phicutoff = abs(phi_max-phi_min)/2.0
psicutoff = abs(psi_max-psi_min)/2.0

In [ ]:
# defining the biasing range for the CV
cv_min = 0.1
cv_max = 10.5 # a bit higher than the upper wall

# upper and lower wall restraints
upper_wall = 10.0*nanometers
lower_wall = 0.1*nanometers

# funnel equation parameters
wall_width = (end_to_end_chain2/2.0)*nanometers   # funnel_wall = wall_width + wall_buffer
wall_buffer = 1.0*nanometers
beta_cent = 1.5             # steepness at inflection point
s_cent = 5.0*nanometers     # location of inflection point

# spring constants for the restraints
k_dihed = 100*kilojoules_per_mole
k_upper = 10000*kilojoules_per_mole/nanometers**2
k_lower = 10000*kilojoules_per_mole/nanometers**2
k_funnel = 10000*kilojoules_per_mole/nanometers**2

In [ ]:
prmtop = AmberPrmtopFile(prmtop_file)
inpcrd = AmberInpcrdFile(inpcrd_file)
pdb = PDBFile(pdb_file)

In [ ]:
# funnel implementation with one CV (distance between COMs of chains, no projections on axes)
start_time = time.time()

system = prmtop.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, hydrogenMass=1.5*amu, 
                             implicitSolvent=OBC2)


dist = CustomCentroidBondForce(2, 'distance(g1,g2)')
dist.addGroup(chain1)
dist.addGroup(chain2)
dist.addBond([0,1])

bv = BiasVariable(dist, cv_min, cv_max, 0.025, False)

# upper and lower wall restraints
upper_wall_rest = CustomCentroidBondForce(3, '0.5*k_upper*max(distance(g1,g2)*sin(angle(g1,g2,g3)) - upper_wall, 0)^2')
upper_wall_rest.addGroup(chain1)
upper_wall_rest.addGroup(first_ca)
upper_wall_rest.addGroup(chain2)
upper_wall_rest.addGlobalParameter('k_upper', k_upper)
upper_wall_rest.addGlobalParameter('upper_wall', upper_wall)
upper_wall_rest.addBond([0,1,2])
system.addForce(upper_wall_rest)

lower_wall_rest = CustomCentroidBondForce(3, '0.5*k_lower*min(distance(g1,g2)*sin(angle(g1,g2,g3)) - lower_wall, 0)^2')
lower_wall_rest.addGroup(chain1)
lower_wall_rest.addGroup(first_ca)
lower_wall_rest.addGroup(chain2)
lower_wall_rest.addGlobalParameter('k_lower', k_lower)
lower_wall_rest.addGlobalParameter('lower_wall', lower_wall)
lower_wall_rest.addBond([0,1,2])
system.addForce(lower_wall_rest)

# adding funnel restraint
funnel_rest = CustomCentroidBondForce(3, '0.5*k_funnel*max(distance(g1,g2)*cos(angle(g1,g2,g3)) - (h/(1+exp(b*(distance(g1,g2)*sin(angle(g1,g2,g3))-s)))+f), 0)^2')
funnel_rest.addGroup(chain1)
funnel_rest.addGroup(first_ca)
funnel_rest.addGroup(chain2)
funnel_rest.addGlobalParameter('k_funnel', k_funnel)
funnel_rest.addGlobalParameter('h', wall_width)
funnel_rest.addGlobalParameter('b', beta_cent)
funnel_rest.addGlobalParameter('s', s_cent)
funnel_rest.addGlobalParameter('f', wall_buffer)
funnel_rest.addBond([0,1,2])
system.addForce(funnel_rest)

# adding torsional restraints
phiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-phi0)-(-k_dihed*cos_phicutoff))', '-k_dihed*cos(theta-phi0)', '-k_dihed*cos_phicutoff'))
phiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
phiforce.addGlobalParameter('phi0', (phi0*np.pi/180.0)*radians)
phiforce.addGlobalParameter('cos_phicutoff', cos(phicutoff*np.pi/180.0))

psiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-psi0)-(-k_dihed*cos_psicutoff))', '-k_dihed*cos(theta-psi0)', '-k_dihed*cos_psicutoff'))
psiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
psiforce.addGlobalParameter('psi0', (psi0*np.pi/180.0)*radians)
psiforce.addGlobalParameter('cos_psicutoff', cos(psicutoff*np.pi/180.0))


for i in range(len(phi_atoms_res_5_25)):
    phiforce.addTorsion(phi_atoms_res_5_25[i][0], phi_atoms_res_5_25[i][1], phi_atoms_res_5_25[i][2], phi_atoms_res_5_25[i][3])
    psiforce.addTorsion(psi_atoms_res_5_25[i][0], psi_atoms_res_5_25[i][1], psi_atoms_res_5_25[i][2], psi_atoms_res_5_25[i][3])
    
system.addForce(phiforce)
system.addForce(psiforce)

# find system forces and put them in separate groups
for i, f in enumerate(system.getForces()):
    f.setForceGroup(i)

# setting up Metadynamics simulation

meta = Metadynamics(system, [bv], 298.15*kelvin, 15.0, 0.8*kilojoule_per_mole, 100)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)

save_freq = 1000

simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('output.dcd', save_freq))
simulation.reporters.append(StateDataReporter(stdout, save_freq, step=True,
        potentialEnergy=True, temperature=True))

steps = 250000000
meta.step(simulation, steps)

print ("simulation time:", time.time() - start_time, "s")

In [ ]:
traj = md.load('output.dcd', top = prmtop_file)

cv_tuple=[]
energies=[]

for crd in traj.xyz: #getting atom coordinates in each frame to make the simulation context
    simulation.context.setPositions(crd)
    cv_tuple.append(meta.getCollectiveVariables(simulation)) #get CV per frame
    for i,f in enumerate(system.getForces()):
        state = simulation.context.getState(getEnergy=True, groups={i})
        energies.append([f, state.getPotentialEnergy()])

fe = []
for i in meta.getFreeEnergy():
    fe.append(i._value)
    
f = open('cv.txt', 'w')
for i in cv_tuple:
    f.write(str(i[0])+'\n') 
f.close()

f2 = open('fe.txt', 'w')
for i in fe:
    f2.write(str(i)+'\n')
f2.close()

f3 = open('total-bias.txt', 'w')
for i in meta._totalBias:
    f3.write(str(i)+'\n')  
f3.close()


n_forces=0
for i,f in enumerate(system.getForces()):
    n_forces+=1

force_types = []
for i in range(0, n_forces):
    force_types.append(str(type(energies[i][0])).split('.')[-1][:-2])


energies_list = []
for frc in force_types:
    energies_list.append([frc, []])

temp_list = [] # make temporary list to slice and store energy values as printed by OpenMM
for i in range(n_forces):
    temp_list.append([])

idx=0
for i in temp_list:
    i.append(energies[idx::n_forces])
    idx+=1

temp_list_2 = []
for i in range(n_forces):
    temp_list_2.append([])

idx = 0
for lst in temp_list_2:
    for i in range(traj.n_frames):
        lst.append(float(str(temp_list[idx][0][i][1]).split(' ')[0]))
    idx+=1

for i in range(n_forces):
    energies_list[i][1].append(temp_list_2[i])

for frce in range(n_forces):
    fle = open('energy%s.txt' %frce, 'w')
    for i in energies_list[frce]:
     if 'Force' in i or 'Motion' in i:
            fle.write(str(i)+'\n')
    else:
        for j in i[0]:
            fle.write(str(j)+'\n')
fle.close()

# for plotting the energies
#t = np.arange(0.0, 1000.0, 0.004)
#for i in range(n_forces):
#    fig,ax = plt.subplots()
#    ax.plot(t, energies_list[i][1][0])
#    ax.set(xlabel='t (ns)', ylabel='Energy (kJ/mol)', title=energies_list[i][0])
#    fig.savefig('energy%s.png' %i)
#    plt.show()